In [2]:
# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys

# identify user's home directory
user_home_dir = expanduser('~')

# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

import botocore  
import boto3
from boto3.session import Session
import cmocean
from collections import Counter
from dask.distributed import Client
import datetime
import fsspec
from gc import get_referents
import json
import numpy as np
from pathlib import Path
from pprint import pprint
import requests
import s3fs
import sys
from sys import getsizeof
import time as time
from types import ModuleType, FunctionType
import xarray as xr
import matplotlib.pyplot as plt
import zarr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import numpy as np
import matplotlib.patches as mpatches

In [3]:
####DASK
from dask.distributed import Client

#  connec to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:36195")
client.ncores
client.restart()

Modify the diffusion

In [ ]:
diffu=ecco.read_llc_to_tiles('/efs_ecco/ECCO/EMU/emu_input_dir/forcing/input_init/','total_diffkr_r009bit11.bin',nk=-1)
diffu[0:10,:,:,:]=0.5*diffu[0:10,:,:,:]
mod_xx_c_diffu = ecco.llc_tiles_to_compact(diffu)
mod_xx_c_diffu.astype('>f4').tofile('/efs_ecco/fjiang/modi_total_diffkr_r009bit11.bin')

Modify the wind stress
Note that the input file cannot contain any NaN values so removed these values!!!

In [ ]:
dataset_num = 12

ecco_v4r5_mon_mean_native_dir = Path('/efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/')

# list sub-directories (one per dataset)
ecco_v4r5_mon_mean_native_dataset_paths = np.sort(list(ecco_v4r5_mon_mean_native_dir.glob('*')))

for i, d in enumerate(ecco_v4r5_mon_mean_native_dataset_paths):
    print(str(i).zfill(3),d)

#print('selected ', ecco_v4r5_mon_mean_native_dataset_paths[dataset_num])
# Select a dataset (the one containing temperature and salinity)
dataset_num = 12
ST_dataset_dir = ecco_v4r5_mon_mean_native_dataset_paths[dataset_num]


# make a list of all of the files in the directory
ST_dataset_files = np.sort(list(ST_dataset_dir.glob('*nc')))
files_to_load = ST_dataset_files
ds_ST = xr.open_mfdataset(ST_dataset_files)


wind_u_trend = - wind_u_trend/52     ##### Make the increment */week

import xarray as xr
import numpy as np

# Assume wind_u_trend has dims (k, j, i) and shape (13, 90, 90)
time = xr.DataArray(np.arange(1358), dims='time')  # shape: (1358,)

# Expand wind_u_trend to have a 'time' dimension (broadcasted)
wind_u_trend_expanded = wind_u_trend.expand_dims(time=time)  # shape: (time, k, j, i)

# Multiply — xarray handles broadcasting automatically now
wind_u_trend_time = wind_u_trend_expanded * time

mod_xx_c = ecco.llc_tiles_to_compact(wind_u_trend_time)
mod_xx_c.astype('>f4').tofile('/efs_ecco/fjiang/modi_xx_tauu.0000000129.data')


000 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/.ipynb_checkpoints
001 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/ATM_SURFACE_TEMP_HUM_WIND_PRES
002 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/ICE_FRONT_FLUX
003 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/ICE_SHELF_FLUX
004 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_3D_MOMENTUM_ADV_TEND_X
005 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_3D_MOMENTUM_ADV_TEND_Y
006 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_3D_MOMENTUM_DISS_TEND_X
007 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_3D_MOMENTUM_DISS_TEND_Y
008 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_3D_MOMENTUM_TEND_X
009 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_3D_MOMENTUM_TEND_Y
010 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_AND_ICE_SURFACE_FW_FLUX
011 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_AND_ICE_SURFACE_HEAT_FLUX
012 /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_AND_ICE_SURFACE_STRESS
013 /efs_ecco/ECCO/V4/r5/netcdf/nativ